#  Install Dependencies

In [78]:
!pip install tensorflow==2.15.0 keras==2.15 tensorflow-hub==0.13.0 tensorflow-text==2.15.0
!pip install tensorflow==2.15.0 keras==2.15 tensorflow-hub==0.13.0 tensorflow-text==2.15.0
!pip install datasets
!pip -q install evaluate
!pip install imbalanced-learn
# !pip install torch torchvision torchaudio
!pip install transformers datasets scikit-learn
!pip install imbalanced-learn

# Import Libraries

In [79]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.layers import Input

import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from transformers import RobertaTokenizer

from sklearn.utils import shuffle


# Tensor Flow

import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# Check if libraries are installed and print their versions
try:
    print(f"TensorFlow Hub Version: {hub.__version__}")
except ImportError:
    print("TensorFlow Hub is not installed.")

try:
    print(f"TensorFlow Text Version: {text.__version__}")
except ImportError:
    print("TensorFlow Text is not installed.")

try:
    print(f"TensorFlow Version: {tf.__version__}")
except ImportError:
    print("TensorFlow is not installed.")

try:
    print(f"pandas Version: {pd.__version__}")
except ImportError:
    print("pandas is not installed.")

try:
    print(f"scikit-learn Version: {train_test_split.__module__.split('.')[0]}")
except ImportError:
    print("scikit-learn is not installed.")

try:
    print(f"scikit-learn Version: {classification_report.__module__.split('.')[0]}")
except ImportError:
    print("scikit-learn is not installed.")

#other usefull

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
# from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

TensorFlow Hub Version: 0.13.0
TensorFlow Text Version: 2.15.0
TensorFlow Version: 2.15.0
pandas Version: 2.2.2
scikit-learn Version: sklearn
scikit-learn Version: sklearn


# Import Github

In [80]:
!git clone https://github.com/tajuar-akash-hub/Datasets.git

fatal: destination path 'Datasets' already exists and is not an empty directory.


# Dataset Location

In [81]:
my_df_location = ('/kaggle/working/Datasets/generated_with_gemini.csv')

# Data preprocessing for Bert

In [82]:
df = pd.read_csv(my_df_location)

df.head(2)


,Generated_using_gemini,index_number,ClaimLegitimacy,fraud_status
0,"In the quaint, sun-kissed hills of New Alishav...",0,Legitimate,0
1,"In the twilight of his years, Mr. Johnson, a 9...",1,Legitimate,0


In [83]:
df.iloc[200]['Generated_using_gemini']  

'In the quaint town of New Thomasberg, an 85-year-old widowed woman named Evelyn embarked on a journey of healing. As a student, her modest income of $26,726.91 presented financial challenges.\n\nOne fateful day, Evelyn sought medical attention from an Orthopedic specialist for a routine procedure. The claim amounted to $9,784.38, a substantial expense for her. With trembling hands, she submitted the claim via paper.\n\nAfter an anxious wait, Evelyn received the news that her claim had been approved. A wave of relief washed over her. The skilled care she had received eased her pain, and the financial burden was lifted.\n\nDespite the challenges she faced, Evelyn remained resilient. Her determination to navigate the healthcare system and prioritize her well-being shone through. And so, her healthcare journey served as a testament to the empathy and support that can light the path toward better health.'

In [84]:
print(df.isnull().sum())

Generated_using_gemini    0
index_number              0
ClaimLegitimacy           0
fraud_status              0
dtype: int64


In [85]:
df = df[['Generated_using_gemini','fraud_status']]

## Train test split

In [86]:

from sklearn.model_selection import train_test_split


# Train-test split: Keep both columns together

train_data, test_data = train_test_split(
    df, stratify=data['fraud_status'], test_size=0.2, random_state=42
)

In [89]:
train_data

,Generated_using_gemini,fraud_status
1361,"In the quaint town of Calvinland, an elderly w...",0
3217,In the bustling town of South Sandramouth resi...,0
2301,"In the quiet town of South Monique, a 69-year-...",0
2100,"On a crisp May morning, an elderly woman named...",0
551,"Sarah, a 49-year-old married woman, was devast...",0
...,...,...
1101,"On a chilly November day, a 30-year-old single...",0
1721,"In the heart of Huntsville, a 67-year-old reti...",0
2939,"In the quaint town of East Jasonstad, a 91-yea...",0
1651,"In the small town of Stevemouth, a retired wom...",0


In [92]:
test_data

,Generated_using_gemini,fraud_status
1776,"In the serene town of Lake Jesseland, amidst t...",0
422,"In the quiet town of Martintown, a 41-year-old...",0
1639,"On August 13th, 2023, an elderly gentleman of ...",0
3424,"On August 25, 2022, a 55-year-old woman sought...",0
2293,"On April 8th, a young man of 28 years old, a s...",0
...,...,...
983,"On December 14, 2023, a 45-year-old married ma...",0
3610,"In the bustling town of Lake Sarahside, amidst...",0
1194,"On a crisp November day in 2023, a 92-year-old...",0
3112,"On October 30, 2022, an 82-year-old widower fi...",0


##  Apply RandomOverSampler to the training data


In [104]:
ros = RandomOverSampler(random_state=42)


# Perform oversampling

train_data_resampled, train_labels_resampled = ros.fit_resample(
    train_data[['Generated_using_gemini']], train_data['fraud_status']
)


# Combine the resampled data back into a DataFrame
train_data_resampled = pd.concat(
    [train_data_resampled['Generated_using_gemini'], train_labels_resampled], axis=1
)


In [105]:
train_data_resampled['fraud_status'].value_counts()

fraud_status
0    2744
1    2744
Name: count, dtype: int64

In [107]:
# Shuffle the oversampled training data

train_data_resampled = shuffle(train_data_resampled, random_state=42)


In [108]:
# Print the distribution after oversampling

from collections import Counter
print("Training distribution after oversampling:", Counter(train_data_resampled['fraud_status']))


Training distribution after oversampling: Counter({1: 2744, 0: 2744})


In [109]:
# Separate features and labels after oversampling

train_texts = train_data_resampled['Generated_using_gemini'].tolist()
train_labels = train_data_resampled['fraud_status'].tolist()


test_texts = test_data['Generated_using_gemini'].tolist()
test_labels = test_data['fraud_status'].tolist()


In [110]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



In [111]:
# Tokenize the training and test sets

train_encodings = tokenizer(
    train_texts, truncation=True, padding=True, max_length=512
)


test_encodings = tokenizer(
    test_texts, truncation=True, padding=True, max_length=512
)

In [112]:
# Convert to Dataset
from datasets import Dataset

def create_dataset(encodings, labels):
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

train_dataset = create_dataset(train_encodings, train_labels)
test_dataset = create_dataset(test_encodings, test_labels)

# Bert Implementation

## Train test split

### Model

In [113]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

### Train Model

In [114]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import evaluate
import numpy as np

# Model Initialization
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir='./bert-results',           # Directory for saving results
    evaluation_strategy="epoch",          # Evaluate at the end of each epoch
    save_strategy="epoch",                # Save model at the end of each epoch
    learning_rate=2e-5,                   # Learning rate
    per_device_train_batch_size=16,       # Training batch size
    per_device_eval_batch_size=16,        # Evaluation batch size
    num_train_epochs=5,                   # Number of training epochs
    weight_decay=0.01,                    # Weight decay
    logging_dir='./logs',                 # Directory for logging
    logging_steps=10,                     # Log every 10 steps
    save_total_limit=2,                   # Keep the last 2 checkpoints
    load_best_model_at_end=True,          # Load the best model at the end of training
    metric_for_best_model="accuracy",     # Metric to select the best model
    fp16=True,                            # Enable mixed precision training
    report_to="none"                      # Disable reporting to external services
)

# Define Metrics
def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "precision": precision_score(labels, predictions, average="weighted"),
        "recall": recall_score(labels, predictions, average="weighted"),
        "f1": f1_score(labels, predictions, average="weighted")
    }

# Trainer Initialization
trainer = Trainer(
    model=model,                  # The model
    args=training_args,           # Training arguments
    train_dataset=train_dataset,  # The training dataset
    eval_dataset=test_dataset,    # The evaluation dataset
    tokenizer=tokenizer,          # Tokenizer
    compute_metrics=compute_metrics  # Custom metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-114-47ec65937c02>:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train the Model

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.080200,0.242855,0.941257,0.931464,0.941257,0.934776
2,0.001700,0.345383,0.948087,0.940636,0.948087,0.942774


In [ ]:
# Evaluate on Test Set

results = trainer.evaluate(eval_dataset=test_dataset)
print("Evaluation Results:", results)


### Load the saved model

In [ ]:
import tensorflow_hub as hub
from keras.models import load_model

# Load the model and specify the custom_objects parameter
model = load_model(
    '/kaggle/working/model_checkpoint_epoch_10.keras',
    custom_objects={'KerasLayer': hub.KerasLayer}
)

print("Model loaded successfully!")



In [118]:
# Evaluate the model on the test dataset
loss, accuracy, precision, recall = model.evaluate(eval_dataset=test_dataset)

# Print the evaluation metrics
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

54/54 [==============================] - 19s 337ms/step - loss: 0.6867 - accuracy: 0.5396 - precision: 0.6371 - recall: 0.1841
Loss: 0.6867239475250244
Accuracy: 0.5396270155906677
Precision: 0.6370967626571655
Recall: 0.18414919078350067
